<a href="https://colab.research.google.com/github/sofials2002/SOFIA/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wage Prediction using Machine Learning

In this notebook we will answer what determines the wage of workers from a predictive perspective.

This example focuses on a sample of Registered Nurses in the US collected during 2017. The hourly wage of a nurse is denoted by $Y$ and $X$ is a vector of nurses' characteristics, e.g., human capital, demographics, job-relevant characteristics. The question that we want to answer is:

- How to use nurses' characteristics, such as education and experience, to best predict wages?


In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import TransformerMixin, RegressorMixin, BaseEstimator
from sklearn.pipeline import make_pipeline

import warnings
warnings.simplefilter('ignore')

## Data

In [2]:
# Read data
path_data = "https://github.com/pabloestradac/causalml-basics/raw/main/data/"
df = pd.read_csv(path_data + 'wages_nurses.csv')
df.tail()

,lwage,female,age,race,children,marital,education,RN_experience,left_nursing,country_ed_US,english_only,military,certificates,labor_union,work_setting,work_situation,level_care,care_specialty,state
5617,3.496087,1,72,white,0,widowed_divorced_sep,ed_assoc,51,left_0,1,1,never_served_mil,0,0,SET_long_term_inpatient,SIT_self_employed,LC_nursing_home,CC_no_patient_care,state_n13
5618,3.678408,1,50,white,0,never_married,ed_msn,14,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_education,CC_emergency_care,state_n24
5619,3.952845,0,36,white,0,currently_married,ed_bsn,4,left_0,1,1,never_served_mil,2,1,SET_hospital,SIT_agency_facility,LC_inpatient,CC_oncology,state_n32
5620,3.957960,1,61,white,0,currently_married,ed_bsn,40,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_inpatient,CC_no_patient_care,state_n37
5621,3.100270,1,38,white,0,currently_married,ed_assoc,3,left_0,1,1,never_served_mil,2,0,SET_hospital,SIT_agency_facility,LC_others,CC_medical_surgical,state_n29


In [3]:
# Outcome and regressors
y = np.log(df['lwage']).values
Z = df.drop(['lwage'], axis=1)
Z.shape

(5622, 18)

Construct a prediction rule for hourly (log) wage $y$, which depends linearly on relevant characteristics $Z$:

$$
y = g(Z) + e
$$

Then, assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$. Thus, we measure the prediction quality of the models via data splitting.


In [4]:
# Obtain train_idx and test_idx to divide y_train and y_test
train_idx, test_idx = train_test_split(np.arange(len(y)), test_size=0.3, random_state=42)
y_train, y_test = y[train_idx], y[test_idx]

## Linear Models

We employ this specification for prediction:

1. Basic Model: $X$ consists of a set of raw regressors



In [5]:
model_base = ('0 + female + age + race + children + marital '
              '+ education + RN_experience + RN_experience**2 + RN_experience**3 + RN_experience**4 '
              '+ left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state')
Zbase = patsy.dmatrix(model_base, Z, return_type='dataframe').values
X_train, X_test = Zbase[train_idx], Zbase[test_idx]

In [6]:
def metrics(X_test, y_test, estimator):
    mse = np.mean((y_test - estimator.predict(X_test))**2)
    semse = np.std((y_test - estimator.predict(X_test))**2) / np.sqrt(len(y_test))
    r2 = 1 - mse / np.var(y_test)
    print(f'{mse:.4f}, {semse:.4f}, {r2:.4f}')
    return mse, semse, r2

results = {}

Let's start by running a simple OLS regression.

In [7]:
# OLS using LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
ypred_ols = lr.predict(X_test)
results['ols'] = metrics(X_test, y_test, lr)

0.0079, 0.0005, 0.2313


In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Lasso
lcv = make_pipeline(StandardScaler(), LassoCV(cv=cv, random_state=42))
lcv.fit(X_train, y_train)
ypred_lcv = lcv.predict(X_test)
results['lassocv'] = metrics(X_test, y_test, lcv)

0.0078, 0.0005, 0.2321


In [9]:
# Ridge
rcv = make_pipeline(StandardScaler(), RidgeCV(cv=cv))
rcv.fit(X_train, y_train)
ypred_rcv = rcv.predict(X_test)
results['ridgecv'] = metrics(X_test, y_test, rcv)

0.0078, 0.0005, 0.2316


In [10]:
# Elastic Net
ecv = make_pipeline(StandardScaler(), ElasticNetCV(cv=cv, random_state=42))
ecv.fit(X_train, y_train)
ypred_ecv = ecv.predict(X_test)
results['elanetcv'] = metrics(X_test, y_test, ecv)

0.0078, 0.0005, 0.2321


Use theoretical-optimal $\lambda$ for lasso.

This is a based on a Python implementation made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy).

Run the following code to install the library:

```python
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess
```

In [12]:
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess

Cloning into 'hdmpy'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 70 (delta 39), reused 52 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 25.30 KiB | 5.06 MiB/s, done.
Resolving deltas: 100% (39/39), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00


In [13]:
import hdmpy

# Wrap the package so that it has the familiar sklearn API
class RLasso(BaseEstimator, RegressorMixin):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    @property
    def coef_(self):
        return np.array(self.rlasso_.est['beta']).flatten()

    @property
    def intercept_(self):
        return np.array(self.rlasso_.est['intercept'])

    def predict(self, X):
        return X @ self.coef_ + self.intercept_

In [14]:
# Optimal lasso
lasso = make_pipeline(StandardScaler(), RLasso(post=False))
lasso.fit(X_train, y_train)
ypred_lasso = lasso.predict(X_test)
results['lasso'] = metrics(X_test, y_test, lasso)

0.0082, 0.0005, 0.2006


In [15]:
# Optimal post-lasso
postlasso = make_pipeline(StandardScaler(), RLasso(post=True))
postlasso.fit(X_train, y_train)
ypred_postlasso = postlasso.predict(X_test)
results['postlasso'] = metrics(X_test, y_test, postlasso)

0.0084, 0.0006, 0.1749


In [ ]:
pd.DataFrame(results).T

,0,1,2
ols,0.007852,0.000513,0.231310
lassocv,0.007843,0.000515,0.232147
ridgecv,0.007849,0.000513,0.231600
elanetcv,0.007844,0.000515,0.232127
lasso,0.008165,0.000533,0.200628
postlasso,0.008428,0.000556,0.174893


## Non-Linear Models

Apply regression trees, random forests, boosted trees and neural nets to estimate the regression function $g(Z)$.

In [16]:
dtr = DecisionTreeRegressor(min_samples_leaf=5)
dtr.fit(X_train, y_train)
ypred_dtr = dtr.predict(X_test)
results['trees'] = metrics(X_test, y_test, dtr)

0.0109, 0.0006, -0.0627


In [17]:
rf = RandomForestRegressor(n_estimators=5000, min_samples_leaf=5)
rf.fit(X_train, y_train)
ypred_rf = rf.predict(X_test)
results['randforest'] = metrics(X_test, y_test, rf)

0.0082, 0.0005, 0.2016


In [20]:
gbf = GradientBoostingRegressor(n_estimators=2500)
gbf.fit(X_train, y_train)
ypred_gbf = gbf.predict(X_test)
results['gradboost'] = metrics(X_test, y_test, gbf)

0.0099, 0.0006, 0.0317


In [32]:
nnet = MLPRegressor(hidden_layer_sizes=(200, 20, ),
                    max_iter=20,
                    alpha=1e-4,
                    learning_rate_init=0.01,
                    verbose=True,
                    random_state=42)

nnet.fit(X_train, y_train)
ypred_nnet = nnet.predict(X_test)
results['neuralnet'] = metrics(X_test, y_test, nnet)

Iteration 1, loss = 7.49166990
Iteration 2, loss = 0.28381746
Iteration 3, loss = 0.12928758
Iteration 4, loss = 0.02008098
Iteration 5, loss = 0.01261349
Iteration 6, loss = 0.00913144
Iteration 7, loss = 0.00665508
Iteration 8, loss = 0.00574887
Iteration 9, loss = 0.00541616
Iteration 10, loss = 0.00518999
Iteration 11, loss = 0.00494119
Iteration 12, loss = 0.00483220
Iteration 13, loss = 0.00474113
Iteration 14, loss = 0.00459127
Iteration 15, loss = 0.00460573
Iteration 16, loss = 0.00457317
Iteration 17, loss = 0.00451058
Iteration 18, loss = 0.00447765
Iteration 19, loss = 0.00433670
Iteration 20, loss = 0.00453871
0.0112, 0.0005, -0.0968


In [34]:
!pip install skorch
import skorch
from torch import nn, optim

arch = nn.Sequential(nn.Linear(X_train.shape[1], 200), nn.ReLU(),
                     nn.Linear(200, 20), nn.ReLU(),
                     nn.Linear(20, 1))
nnet_early = skorch.NeuralNetRegressor(arch, lr=0.01, batch_size=10,
                                       max_epochs=100,
                                       optimizer=optim.Adam,
                                       callbacks=[skorch.callbacks.EarlyStopping()])
nnet_early.fit(X_train.astype(np.float32), y_train.reshape(-1, 1).astype(np.float32))
ypred_nnet_early = nnet_early.predict(X_test.astype(np.float32)).flatten()
results['neuralnet_early'] = metrics(X_test.astype(np.float32),
                                y_test.reshape(-1, 1).astype(np.float32), nnet_early)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 4.7 MB/s eta 0:00:00
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.0486        0.0179  0.9148
      2        0.0173        0.0186  0.7591
      3        0.0121        0.0170  1.0163
      4        0.0104        0.0161  1.1240
      5        0.0096        0.0132  1.1511
      6        0.0091        0.0109  0.9321
      7        0.0086        0.0110  0.7904
      8        0.0083        0.0109  0.8464
      9        0.0080        0.0103  0.7813
     10        0.0080        0.0097  0.8132
     11        0.0079        0.0093  0.8537
     12        0.0078        0.0093  0.9027
     13        0.0076        0.0092  0.8104
     14        0.0077        0.0093  0.8263
     15        0.0076        0.0102  0.8426
     16        0.0076        0.0093  0.7828
     17        0.0076        0.0093  0.7631
Stopping since valid_loss has not improved in the last 5 epochs.
0.0085, 0.0005, 0.1

In [35]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df.sort_values('MSE', ascending=True)

,MSE,S.E. MSE,Rsq
lassocv,0.007843,0.000515,0.232147
elanetcv,0.007844,0.000515,0.232127
ridgecv,0.007849,0.000513,0.231600
ols,0.007852,0.000513,0.231310
randforest,0.008155,0.000499,0.201607
lasso,0.008165,0.000533,0.200628
postlasso,0.008428,0.000556,0.174893
neuralnet_early,0.008530,0.000536,0.164941
gradboost,0.009890,0.000601,0.031730
trees,0.010856,0.000614,-0.062744


## Stacking

In the final step, build a prediction model by combining the strength of each model.
$$
g(z) = \sum_{k=1}^K \alpha_k g_k(z)
$$
where $g_k$'s denote our prediction rules from the table above and the $\alpha_k$'s are the corresponding weights.

In [39]:
np.stack([ypred_ols, ypred_lcv, ypred_rcv, ypred_ecv, ypred_lasso, ypred_postlasso,
                   ypred_dtr, ypred_rf, ypred_gbf, ypred_nnet, ypred_nnet_early], axis=-1).shape

(1687, 11)

In [40]:
method_name = ['OLS', 'CV Lasso', 'CV Ridge', 'CV ElasticNet', 'Lasso', 'Post-Lasso OLS',
               'Decision Tree', 'Random Forest', 'Boosted Forest', 'Neural Net', 'Neural Net (early stopping)']

ypreds = np.stack([ypred_ols, ypred_lcv, ypred_rcv, ypred_ecv, ypred_lasso, ypred_postlasso,
                   ypred_dtr, ypred_rf, ypred_gbf, ypred_nnet, ypred_nnet_early], axis=-1)


stack_ols = LinearRegression().fit(ypreds, y_test)
pd.DataFrame({'weight': stack_ols.coef_}, index=method_name).round(2)

,weight
OLS,1.22
CV Lasso,193.86
CV Ridge,-0.82
CV ElasticNet,-193.54
Lasso,-0.31
Post-Lasso OLS,-0.02
Decision Tree,-0.01
Random Forest,0.48
Boosted Forest,-0.03
Neural Net,-0.12


In [41]:
# Calculate the test sample MSE and Rsq
# We should have left out a third sample to validate the performance of the stacked model
mse = np.mean((y_test - stack_ols.predict(ypreds))**2) #predicting using a weighted average of the other predictions.
r2 = 1 - mse / np.var(y_test)
print(f'MSE: {mse:.4f}, R^2: {r2:.4f}')

MSE: 0.0075, R^2: 0.2665


## Pipelines

We can also do it in a more sklearn way, by defining a formula transformer and corresponding pipelines

In [48]:
class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula):
        self.formula = formula

    def fit(self, X, y=None):
        mat = patsy.dmatrix(self.formula, X, return_type='matrix')
        self.design_info = mat.design_info
        return self

    def transform(self, X, y=None):
        return patsy.build_design_matrices([self.design_info], X)[0]

In [49]:
base = FormulaTransformer(model_base)
#methods =

In [ ]:
results = {}
ypreds = np.zeros((len(test_idx), len(methods)))  # test predictions used for stacking

for it, (name, estimator) in enumerate(methods):
    # Fit the estimator and predict

    ypreds[:, it] =
    results[name] = metrics(Z.iloc[test_idx], y[test_idx], estimator)

In [ ]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df

1. Partition the data in k-folds

2. For each fold, train each of the estimators in the `methods` parameter on all the data outside of the fold and then predict on the data in the fold

3. Train a `final_estimator` predicting the true outcome using the out-of-fold predictions of each method as features

4. All the base estimators are re-fitted on all the data and the final predictor will first predict based on each fitted based estimator and then aggregate based on the fitted `final_estimator`


In [ ]:
# Stacking with sklearn
stack =

In [ ]:
# Weights of the final estimator
pd.DataFrame({'weight': stack.final_estimator_.coef_}, index=[name for name, _ in methods])

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], stack)

## AutoML

In [42]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 6.5 MB/s eta 0:00:00


In [ ]:
from flaml import AutoML

automl = make_pipeline(base, AutoML(task='regression', time_budget=60, early_stop=True,
                                    eval_methods='cv', n_splits=5, metric='r2', verbose=3))

automl.fit(X_train, y_train)

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)

In [ ]:
# Stacking with flaml
automl =

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)